In [1]:
import pandas as pd
import numpy as np

In [2]:
#main raw file which includes all Items
fao_raw=pd.read_csv('FAOSTAT_data_producer_prices (only value data).csv')

In [3]:
fao_raw.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Months Code,Months,Unit,Value
0,PP,Producer Prices,2,Afghanistan,5530,Producer Price (LCU/tonne),515,Apples,1993,1993,7021,Annual value,LCU,6100.0
1,PP,Producer Prices,2,Afghanistan,5530,Producer Price (LCU/tonne),44,Barley,1993,1993,7021,Annual value,LCU,6300.0
2,PP,Producer Prices,2,Afghanistan,5530,Producer Price (LCU/tonne),56,Maize,1993,1993,7021,Annual value,LCU,16000.0
3,PP,Producer Prices,2,Afghanistan,5530,Producer Price (LCU/tonne),568,"Melons, other (inc.cantaloupes)",1993,1993,7021,Annual value,LCU,19000.0
4,PP,Producer Prices,2,Afghanistan,5530,Producer Price (LCU/tonne),116,Potatoes,1993,1993,7021,Annual value,LCU,2800.0


In [4]:
len(fao_raw)

148619

In [5]:
#delete LCL value
fao_raw=fao_raw[fao_raw['Unit']!='LCU']

In [6]:
len(fao_raw)

114253

In [7]:
#delete Maize, Green value
fao_raw=fao_raw[fao_raw['Item Code']!=446]
len(fao_raw)

112452

In [16]:
fao_raw.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Months Code,Months,Unit,Value
126,PP,Producer Prices,2,Afghanistan,5531,Producer Price (SLC/tonne),515,Apples,1993,1993,7021,Annual value,SLC,6100.0
127,PP,Producer Prices,2,Afghanistan,5531,Producer Price (SLC/tonne),44,Barley,1993,1993,7021,Annual value,SLC,6300.0
128,PP,Producer Prices,2,Afghanistan,5531,Producer Price (SLC/tonne),56,Maize,1993,1993,7021,Annual value,SLC,16000.0
129,PP,Producer Prices,2,Afghanistan,5531,Producer Price (SLC/tonne),568,"Melons, other (inc.cantaloupes)",1993,1993,7021,Annual value,SLC,19000.0
130,PP,Producer Prices,2,Afghanistan,5531,Producer Price (SLC/tonne),116,Potatoes,1993,1993,7021,Annual value,SLC,2800.0


In [8]:
len(fao_raw['Item'].unique())

23

In [9]:
#shortName=pd.read_csv('fao/country_shortname_V1.csv',usecols=[0, 1,2])
shortName=pd.read_excel('fao/country_short_name_with_percantege.xlsx',usecols=[0,1,2,6,7,8,9])

In [160]:
for u in fao_raw['Item'].unique():
    print(u)

Apples
Barley
Maize
Melons, other (inc.cantaloupes)
Potatoes
Rice, paddy
Wheat
Watermelons
Peaches and nectarines
Pears
Plums and sloes
Sugar cane
Sunflower seed
Rye
Soybeans
Cucumbers and gherkins
Pumpkins, squash and gourds
Sugar beet
Rapeseed
Avocados
Mangoes, mangosteens, guavas
Cocoa, beans
Oil, palm


In [99]:
################### ITEM LIST FOR EMPTY DATAFRAME ##########################
item_list=[]
for item in fao_raw['Item'].unique():    
    item_name=item
    item_name=item_name.replace(',','_')
    item_name=item_name.replace(' ','_')
    item_name=item_name.replace('__','_')
    #new_name=item_name.split('_')[0:2]
    #item_name='_'.join(new_name)
    if item_name[0:8].split('_')[0]=='Sugar':
        item_name=item_name[0:7]
    else:
        item_name=item_name[0:8].split('_')[0]
    item_list.append(item_name+'_i')
    item_list.append(item_name+'_s')
    item_list.append(item_name+'_u')

In [101]:
output_folder='fao_price'
#for i in fao_raw['Item Code'].unique():
for y in fao_raw['Year'].unique():
    # with this for loop, we export each product from fao excel file
    # y means indiviual year
    
    #create dataframe according to year
    byYear_target_item=fao_raw[fao_raw['Year']==y]    
    #create empty dataframe for year dataframe 
    header_list=['Domain Code','Domain','Area Code','Area','Element Code','Element']
    # we use item list, we create three column for each product. for SLC, USD and index value
    yearly_df=pd.DataFrame(columns=header_list+item_list)
    for i in byYear_target_item['Area Code'].unique():
        # get each country stastic into dataframe
        byCountry_target=byYear_target_item[byYear_target_item['Area Code']==i]
        country_df=pd.DataFrame(columns=header_list)

        for u in byCountry_target['Unit'].unique():
            byUnit=byCountry_target[byCountry_target['Unit']==u]
            # create dataframe with country fundemental inforamtion: Area Code, Country, Area
            target_df=byCountry_target.iloc[:1,0:6]
            #target_df=target_df.reindex(columns=header_list+item_list,copy=False)
            #add year information into row
            target_df['Year']=int(y)   
            for item in byUnit['Item Code'].unique():
                item_name=byCountry_target[byCountry_target['Item Code']==item].Item.values[0]
                item_name=item_name.replace(',','_')
                item_name=item_name.replace(' ','_')
                item_name=item_name.replace('__','_')
                if item_name[0:8].split('_')[0]=='Sugar':
                    item_name=item_name[0:7]
                else:
                    item_name=item_name[0:8].split('_')[0]
                
                if type(u) ==np.float:
                    item_name=item_name+'_i'
                elif u=='SLC':
                    item_name=item_name+'_s'
                else:
                    item_name=item_name+'_u'
                    
                item_df=byCountry_target[byCountry_target['Item Code']==item]
            

                target_df[item_name]=item_df['Value'].values[0]
                
            country_df=country_df.append(target_df)
        # pivot table for each country
        piv_country=country_df.pivot_table(index=['Domain Code','Domain','Area Code','Area','Element Code','Element'])
        piv_country=piv_country.reset_index()
        piv_country=piv_country.reindex(columns=header_list+item_list,copy=False)
        yearly_df=yearly_df.append(piv_country)
        #yearly_df=yearly_df.reindex(columns=header_list+item_list,copy=False)
        yearly_df.reset_index(inplace=True,drop=True)
        yearly_df['Area Code']=yearly_df['Area Code'].astype('str')
        shortName['Area Code']=shortName['Area Code'].astype('str')
    

        # we get country short name info from shortName csv file and merge with pivot_product
        last_product=pd.merge(shortName,yearly_df,left_on='Area Code',right_on='Area Code',how='inner')
        # drop duplicate column and rename the Area column
        last_product.drop(columns=['Area_y'],inplace=True)
        #last_product.drop(columns=['Country_y'],inplace=True)
        last_product.rename(columns={'Area_x':'Area'},inplace=True)
        #last_product.rename(columns={'Country_x':'Country'},inplace=True)
        last_product.columns=last_product.columns.astype(str)
        
        index_list=[]
        for index,row in last_product.iterrows():
            #if row include seperation information code starts
            if not pd.isna(row['Country_split_(by_area_code)']):            
                split_by_area_code=row['Country_split_(by_area_code)'].split(',')
                split_year=int(row['Split_Year'])
                #print(split_by_area_code)
                if int(y)<int(split_year):                                             
                    for index2,area_code in enumerate(split_by_area_code):                        
                        pd.options.mode.chained_assignment = None 
                        target_row=shortName[shortName['Area Code']==str(area_code)]
                        df_row=row.to_frame()
                        for index3,value in enumerate(row[11:],11):
                            #print(area_code)
                            new_value=float(row[index3])
                            target_row.loc[:,'Year']=int(y)
                            target_row.loc[:,df_row.index[index3]]=new_value
                        
                        # add the new row(country) into last product
                        target_row=target_row.reindex(columns=list(last_product.columns),copy=False)
                        last_product=last_product.append(target_row)
                    index_list.append(index)    
        #reset last product index
        last_product=last_product.reset_index(drop=True)    
        for drop_index in index_list:
            last_product.drop(drop_index, inplace=True)
              
    
               
    last_product.to_csv(f'{output_folder}/fao_price_{str(y)}.csv',index=False)
    
    
    

In [102]:
last_product.columns

Index(['Area', 'Country', 'Area Code', 'Split_Year', 'What_happened',
       'Country_split_(by_area_code)', 'Percentages', 'Domain Code', 'Domain',
       'Element Code', 'Element', 'Apples_i', 'Apples_s', 'Apples_u',
       'Barley_i', 'Barley_s', 'Barley_u', 'Maize_i', 'Maize_s', 'Maize_u',
       'Melons_i', 'Melons_s', 'Melons_u', 'Potatoes_i', 'Potatoes_s',
       'Potatoes_u', 'Rice_i', 'Rice_s', 'Rice_u', 'Wheat_i', 'Wheat_s',
       'Wheat_u', 'Watermel_i', 'Watermel_s', 'Watermel_u', 'Peaches_i',
       'Peaches_s', 'Peaches_u', 'Pears_i', 'Pears_s', 'Pears_u', 'Plums_i',
       'Plums_s', 'Plums_u', 'Sugar_c_i', 'Sugar_c_s', 'Sugar_c_u',
       'Sunflowe_i', 'Sunflowe_s', 'Sunflowe_u', 'Rye_i', 'Rye_s', 'Rye_u',
       'Soybeans_i', 'Soybeans_s', 'Soybeans_u', 'Cucumber_i', 'Cucumber_s',
       'Cucumber_u', 'Pumpkins_i', 'Pumpkins_s', 'Pumpkins_u', 'Sugar_b_i',
       'Sugar_b_s', 'Sugar_b_u', 'Rapeseed_i', 'Rapeseed_s', 'Rapeseed_u',
       'Avocados_i', 'Avocados_s', 'Avo

In [96]:
a=['Sugar_beet','Sugar_cane']

In [98]:
a[0][0:7]

'Sugar_b'

In [ ]:
item_name=item_name[0:8].split('_')[0]
